## Plot Wflow results

**HydroMT** provides a simple interface to model results from which we can make beautiful plots:

- Results layers are saved to the model `results` component as a dictionnary of `xarray.Dataset` or `xarray.DataArray`.

These plots can be useful to analyze the model results or also compare model runs with different settings (different precipitation source or different parameters values).

### Load dependencies

In [ ]:
import matplotlib.pyplot as plt
import hydromt
from hydromt_wflow import WflowModel

### Read the model run(s) results

The **wflow_piave_subbasin** model was run using the default global data sources of the hydromt_wflow plugin. The different variables to save after the wflow were set in a separate wflow configuration file: *wflow_sbm_results.toml*.

A second run of the model was also done, where the KsatHorFrac parameter of wflow was set to 10 (instead of the default 100 value) using an alternative configuration file: *wflow_sbm_results2.toml*.

We will use the below `runs` dictionnary to define the model run(s) we want to read and some settings for plotting. If you want to plot and compare several runs together, you can simply add them to the `runs` dictionnary.

In [ ]:
# Dictionary listing the different wflow models and runs to compare, including plotting options
runs = {
    "run1": {
        "longname": "default",
        "color": "blue",
        "root": "wflow_piave_subbasin",
        "config_fn": "wflow_sbm_results.toml",
    },
    "run2": {
        "longname": "KsatHorFrac10",
        "color": "green",
        "root": "wflow_piave_subbasin",
        "config_fn": "wflow_sbm_results2.toml",
    },
}
mainrun = "run1"

In [ ]:
# Initialize the different model run(s)
for r in runs:
    run = runs[r]
    mod = WflowModel(root=run["root"], mode="r+", config_fn=run["config_fn"])
    runs[r].update({"mod": mod})

Wflow can save different types of outputs (netcdf gridded output, netcdf scalar netcdf, csv scalar timeseries) that are also reflected in the organisation of the HydroMT `results` component:

* a **"output" xarray.Dataset** for the gridded netcdf file (output section of the TOML)
* a **"netcdf" xarray.Dataset** for the netcdf point timeseries file (netcdf section of the TOML)
* different **hydromt.GeoDataArrays for the csv file** , one per column (csv section and csv.column sections of the TOML). The xy coordinates are the coordinates of the station or of the representative point of the subcatch/area. The variable name in the GeoDataArray corresponds to the csv header attribute or header_map when available.

Below you can see how to access to the results of *run1* and its contents:

In [ ]:
mod1 = runs["run1"]["mod"]
mod1.results

### Read observations

You can also use HydroMT to read observations data in order to analyze your model results.
Here a **fictional** observations timeseries was prepared for the gauges_grdc locations. 

In [ ]:
# Discharge data
timeseries_fn = "gauges_observed_flow.csv"  # observed discharge timeseries
name = "gauges_grdc"  # gauges locations in staticgeoms
stationID = "grdc_no"  # column name in staticgeoms containing the stations IDs

# Read the observations data
# read timeseries data and match with existing gdf
gdf = runs[mainrun]["mod"].geoms[name]
gdf.index = gdf[stationID]
da_ts = hydromt.io.open_timeseries_from_table(timeseries_fn, name=name, sep=";")
da = hydromt.vector.GeoDataArray.from_gdf(gdf, da_ts, index_dim="index")

obs = da
obs

### Plot model results

Here we plot the different model results for the gauges_grdc locations.

In [ ]:
# Plotting options
# select the gauges_grdc results (name in csv column of wflow results to plot)
result_name = "river_q_gauges_grdc"

# selection of runs to plot (all or a subset)
runs_subset = ["run1", "run2"]

In [ ]:
# Plots
from hydromt.stats import skills as skillstats

station_ids = list(runs[mainrun]["mod"].results[result_name].index.values)

for i, st in enumerate(station_ids):
    n = 2
    fig, axes = plt.subplots(n, 1, sharex=True, figsize=(15, n * 4))
    axes = [axes] if n == 1 else axes

    # Discharge
    obs_i = obs.sel(index=st)
    obs_i.plot.line(ax=axes[0], x="time", label="obs", color="black")

    for r in runs_subset:
        run = runs[r]
        run_i = run["mod"].results[result_name].sel(index=st)
        # Stats
        nse_i = skillstats.nashsutcliffe(run_i, obs_i).values.round(2)
        kge_i = skillstats.kge(run_i, obs_i)["kge"].values.round(2)
        labeltxt = f"{run['longname']}, NSE: {nse_i}, KGE: {kge_i}"
        run_i.plot.line(
            ax=axes[0],
            x="time",
            label=labeltxt,
            color=f"{run['color']}",
            linestyle="--",
        )

    axes[0].set_title(f"Simulated discharge at station {st}")
    axes[0].set_ylabel("Discharge [m3/s]")
    axes[0].legend()

    # Cumulative Discharge
    obs_i = obs.sel(index=st)
    obs_i.cumsum().plot.line(ax=axes[1], x="time", label="obs", color="black")

    for r in runs_subset:
        run = runs[r]
        run_i = run["mod"].results[result_name].sel(index=st)
        run_i.cumsum().plot.line(
            ax=axes[1],
            x="time",
            label=f"{run['longname']}",
            color=f"{run['color']}",
            linestyle="--",
        )

    axes[1].set_title(f"Cumulative discharge at station {st}")
    axes[1].set_ylabel("Cumulative Discharge [m3/s]")
    axes[1].legend()

You can see on the discharge plots legends that some statistical criteria were computed using the fictional observations and the model runs results.

These statistics were computed using the **stats** module of HydroMT. You can find the available statisctics functions in the [documentation](https://deltares.github.io/hydromt/stable/guides/advanced_user/methods_stats.html).

And finally once the `results` are loaded, you can use them to derive more statistics or plots to further analyze your model.